In [1]:
import pyarrow.parquet as pq
import pyarrow as pa
import ujson
import numpy as np
from rich import progress

# 1. 处理预训练阶段数据
## 1.1 处理Wiki数据

In [2]:
origin_wiki_file = './data/wiki.simple.txt'

liness = []
with open(origin_wiki_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

In [3]:
lines[0:5]

['数学：\n',
 '数学，是研究数量、结构以及空间等概念及其变化的一门学科，属于形式科学的一种。数学利用抽象化和逻辑推理，从计数、计算、量度、对物体形状及运动的观察发展而成。数学家们拓展这些概念，以公式化新的猜想，以及从选定的公理及定义出发，严谨地推导出一些定理。\n',
 '基础数学的知识与运用是生活中不可或缺的一环。对数学基本概念的完善，早在古埃及、美索不达米亚及古印度历史上的古代数学文本便可观见，而在古希腊那里有更为严谨的处理。从那时开始，数学的发展便持续不断地小幅进展，至16世纪的文艺复兴时期，因为新的科学发现和数学革新两者的交互，致使数学的加速发展，直至今日。数学并成为许多国家及地区的教育中的一部分。\n',
 '数学在许多领域都有应用，包括科学、工程、医学、经济学和金融学等。数学对这些领域的应用通常被称为应用数学，有时亦会激起新的数学发现，并导致全新学科的发展，例如物理学的实质性发展中建立的某些理论激发数学家对于某些问题的不同角度的思考。数学家也研究纯粹数学，就是数学本身的实质性内容，而不以任何实际应用为目标。许多研究虽然以纯粹数学开始，但其过程中也发现许多可用之处。\n',
 '词源。\n']

In [4]:
len(lines)

10103707

合并词条和内容

In [5]:
items, content = [], []
key_word, kw_line_idx = '', 0
content_start = False  # 词条内容开始标记

bos_token, eos_token = '[BOS]', '[EOS]' 
for i, line in enumerate(lines):
    
    line_strip = line.strip()

    # 词条以冒号`：`结尾
    if len(line_strip) > 0 and line_strip[-1] in (':', '：'):
        key_word = ''.join(line_strip[: -1])
        kw_line_idx = i 
        continue
    
    # 词条key_word在下一行，则合并上个词条并保存
    if i == kw_line_idx + 1 and key_word in line_strip or i == len(lines) - 1:
        txt = ''.join(content)

        if len(txt) > 0:
            items.append(f"{txt}{eos_token}")
            
        content = []
        content.append(f"{key_word}：")
    
    content.append(line)


In [6]:
len(items)

1008067

In [7]:
items[20]

'地质学：地质学（法语、德语：Geologie；拉丁语、西班牙语：Geologia；源于希腊语，γῆ，和，λoγία）是对地球的起源、历史与结构进行研究的学科。主要研究地球的物质组成、内部构造、外部特征、各圈层间的相互作用和演变历史。在现阶段，由于观察、研究条件的限制，主要以岩石圈为研究对象，并涉及水圈、大气圈、生物圈和岩石圈下更深的部分，以及涉及其他行星和卫星的太空地质学。\n地球历史学。\n-zh-hant：zh-hans\n地质时间尺度涵盖了整个地球历史。其起点开始于最早的太阳系材料生成时，约在4。567Ga，地球的出现约在4。54，Ga，一开始称为冥古宙，最年轻的地质年代则是全新世。\n地质年代比例。\n如果把地球诞生到现在的大约45亿年缩小到1年，则人类（Homo，apiens存在的时间只有极短的两分钟。\n显生宙。\n地质学学史。\n对地球的物质成分的研究最早至少可以追溯到古希腊泰奥弗拉斯托斯的著作《论岩石》Peri，Lithon）在古罗马时期，老普林尼详细的描述了常用的一些矿物和金属，还正确的解释了琥珀的来源。\n一些现代学者（如）认为，现代地质学开始于中世纪伊斯兰世界。比鲁尼就是最早的之一，他的著作有最早的叙述的文章，提出了印度次大陆曾经是海洋的假设。伊斯兰学者伊本·西那对山脉的形成，地震的原因，以及其他一些现代地理学的论题给出了详细的解释，这些内容为日后地质学的发展提供了基础。在中国，博学家沈括（1031–1095）提出陆地形成的假说。他在一个离海洋几百公里远的山中的看到，在一个地质地层里有贝壳类生物化石。由此他推论，陆地是由山脉的侵蚀和淤泥的沉积所形成的。\n很早以前，地质学的知识比较零星分散。关于这方面的知识，如从地中开采金属、黏土、煤和盐的一些知识，早已为矿工和有关的人们所知晓，而自然哲学家们则大都脱离这些实践，独立形成自己的思辨性的地质理论。\n地质学在18世纪开始成为一门独立的科学，并在19世纪早期达到成熟阶段。\n1790年至1830年这一段时期被称为“地质学的英雄时代”在这个时期，在考察岩层顺序以及岩层所含矿物和化石上，人们做了大量工作。在现阶段，由于观察、研究条件的限制，主要以岩石圈为研究对象，并涉及水圈、大气圈、生物圈和岩石圈下更深的部位，以及涉及其他行星和卫星的太空地质学。\n。工作方法的一大进步表现在用根据化石内容来进行岩层分

将Wiki数据合并为长度固定的行

In [8]:
def split_txt_cropus_to_chunk_data(texts: list[str], batch_size: int=512 ** 2, max_len: int=320, window_size: int = 32) -> list[str]:
    
    buffer, buffer_len = [], 0
    chunk_data = []

    for i, line in enumerate(texts):
        buffer_len += len(line)
        buffer.append(line)

        if buffer_len >= batch_size or i == len(texts) - 1:
            buffer_txt = ''.join(buffer)
            
            # - window_size为滑动窗口，这样每个窗口都包含有window_size个上文
            for i in range(0, len(buffer_txt), max_len - window_size):

                chunk_data.append(''.join(buffer_txt[i: i + max_len]))
            
            buffer, buffer_len = [], 0
    
    return chunk_data

In [9]:
chunk_data = split_txt_cropus_to_chunk_data(items)
print(len(chunk_data))

2258333


In [10]:
chunk_data[0: 3]

['数学：数学，是研究数量、结构以及空间等概念及其变化的一门学科，属于形式科学的一种。数学利用抽象化和逻辑推理，从计数、计算、量度、对物体形状及运动的观察发展而成。数学家们拓展这些概念，以公式化新的猜想，以及从选定的公理及定义出发，严谨地推导出一些定理。\n基础数学的知识与运用是生活中不可或缺的一环。对数学基本概念的完善，早在古埃及、美索不达米亚及古印度历史上的古代数学文本便可观见，而在古希腊那里有更为严谨的处理。从那时开始，数学的发展便持续不断地小幅进展，至16世纪的文艺复兴时期，因为新的科学发现和数学革新两者的交互，致使数学的加速发展，直至今日。数学并成为许多国家及地区的教育中的一部分。\n数学在许多领域都有应用，包括科学、工程、医学、',
 '教育中的一部分。\n数学在许多领域都有应用，包括科学、工程、医学、经济学和金融学等。数学对这些领域的应用通常被称为应用数学，有时亦会激起新的数学发现，并导致全新学科的发展，例如物理学的实质性发展中建立的某些理论激发数学家对于某些问题的不同角度的思考。数学家也研究纯粹数学，就是数学本身的实质性内容，而不以任何实际应用为目标。许多研究虽然以纯粹数学开始，但其过程中也发现许多可用之处。\n词源。\n西方语言中“数学”一词源自于古希腊语的（其有“学习”学问”科学”还有个较狭义且技术性的意思－"数学研究"即使在其语源内。其形容词（意思为"和学习有关的"或"用功的"亦会被用来指"数学的"其在英语中表面上的复数形式，及在法语中的表面复数形式\'可溯至拉丁',
 '其在英语中表面上的复数形式，及在法语中的表面复数形式\'可溯至拉丁文的中性复数\'由西塞罗译自希腊文复数（此一希腊语被亚里士多德拿来指"万物皆数"的概念。\n汉字表示的"数学"一词大约产生于中国宋元时期。多指象数之学，但有时也含有今天上的数学意义，例如，秦九韶的《数学九章》永乐大典》记，即《数书九章》也被宋代周密所著的《癸辛杂识》记为《数学大略》数学通轨》明代柯尚迁著）数学钥》清代杜知耕著）数学拾遗》清代丁取忠撰）直到1939年，经过中国数学名词审查委员会研究“算学”与“数学”两词的使用状况后，确认以“数学”表示今天意义上的数学含义。\n历史。\n数学有着久远的历史。它被认为起源于人类早期的生产活动：中国古代的六艺之一就有"数"数学一词在西方']

In [11]:
tb = pa.Table.from_arrays([chunk_data], names=['text'])
# compression='GZIP'
pq.write_table(table=tb, where='./data/wiki_chunk_320_2.2M.parquet', row_group_size=50000, data_page_size=50000, )

In [12]:
# wiki_samples = np.random.choice(chunk_data, size=100_0000).tolist()
wiki_samples = chunk_data[0: 10_0000]

In [13]:
len(wiki_samples)

100000

In [14]:
wiki_samples[-5:]

['熔化神像来铸剑、剪下女人的头发用来制作绳子。迦太基在2个月内，制造了8000个防盾、1。8万把剑、3万支矛、6万个石弩，并在内港建立了一支120艘船的舰队。\n迦太基人坚守孤城达3年之久，多次击退了罗马的进攻。公元前147年，西庇阿·艾米里安（即小西庇阿）被选举为罗马的执政官和远征军司令，得到罗马士兵的拥护。不久，罗马军队利用云梯登城成功。迦太基虽经长久围城却毫不屈服，仍与罗马展开巷战；于是，西庇阿下令放火焚烧已攻下的街巷，许多迦太基人被活活烧死。迦太基城烧了17天，最后，50万人口只剩下5。5万人，他们才向罗马军队投降。迦太基将军哈斯德鲁巴请求投降，虽得到西庇阿准许，但哈斯德鲁巴的妻子斥责丈夫"胆小卑鄙"带著几个儿子投火自杀。残存的',
 '哈斯德鲁巴的妻子斥责丈夫"胆小卑鄙"带著几个儿子投火自杀。残存的迦太基人被卖作奴隶，城市被罗马军团洗劫。\n结果。\n这次战争并无和约，因为迦太基国家已不复存在；乌提卡及其他曾经协助过罗马的非洲城市皆获自由，成为罗马保护国，迦太基的剩余领土则被罗马并吞，成为阿非利加行省。许多迦太基人在攻城的过程中饿死，还有很多人在战斗的最后六天中死去。当整场战争结束时只剩下5万名迦太基居民，这些人口只占战前的一小部份。迦太基被毁灭后，其居民依照当时的习惯罗马卖为奴隶。有一个盛传的说法，称罗马军犁过该地并且洒盐，其实基本可以确定这是一则十九世纪时才有的传言。当代的报告显示，迦太基附近的土地被宣布为，由当地人、罗马人与其他义大利的农夫们分配。北非成为罗马重',
 '宣布为，由当地人、罗马人与其他义大利的农夫们分配。北非成为罗马重要的谷物来源，而罗马统治下的迦太基是这些货物的重要转运枢纽。\n许多重要的布匿城市，例如毛里塔尼亚一带的，由罗马人接管并重建；一些重建的都市包含瓦卢比利斯与莫加多尔。以瓦卢比利斯为例，这个重要的镇在罗马征服的土地中座落于最西部的边界。他位于先前的布匿聚居地，但那个聚居地也建筑在一个早期的新石器时代居住地上。是在攻城战开始时倒向罗马的城市，最后变成了罗马阿非利加行省的首府。一个世纪之后，迦太基的所在地由尤利乌斯·凯撒重建为罗马城市，往后在罗马帝国时期成为帝国在非洲的主要都市之一。\n1985年2月5日，罗马市长与迦太基市长签署了一项象征性的和平条约，此时距离第三次布匿战争已过',
 '迦太基市长签署了一项象征性的和平条约

##  1.2 处理百度百科数据库

In [1]:
import pyarrow.parquet as pq
import pyarrow as pa
import ujson
from unicodedata import normalize

In [2]:
def split_txt_cropus_to_chunk_data(texts: list[str], batch_size: int=512 ** 2, max_len: int=320, window_size: int = 32) -> list[str]:
    
    buffer, buffer_len = [], 0
    chunk_data = []

    for i, line in enumerate(texts):
        buffer_len += len(line)
        buffer.append(line)

        if buffer_len >= batch_size or i == len(texts) - 1:
            buffer_txt = ''.join(buffer)
            
            # - window_size为滑动窗口，这样每个窗口都包含有window_size个上文
            for i in range(0, len(buffer_txt), max_len - window_size):

                chunk_data.append(''.join(buffer_txt[i: i + max_len]))
            
            buffer, buffer_len = [], 0
    
    return chunk_data

In [3]:
bd_baike_563w_file = './data/563w_baidubaike.json'
baike_items = []
eos_token = '[EOS]' 
batch_size, batch_cnt = 100_0000, 0

In [4]:
with open(bd_baike_563w_file, 'r', encoding='utf-8') as f:
    line = f.readline()
    line = normalize('NFKC', line)
    item = ujson.loads(line)
    print(item)

{'title': '红色食品', 'summary': '红色食品是指食品为红色、橙红色或棕红色的食品。科学家认为,多吃些红色食品可预防感冒。红色食品有红柿椒、西红柿、胡萝卜、红心白薯、红果(山楂)、红苹果、草莓、红枣、老南瓜、红米、柿子等。 有治疗缺铁性贫血和缓解疲劳的作用,对乳腺癌等肿瘤疾病有防治作用,给人以兴奋感,有增加食欲,光洁皮肤,增强表皮细胞再生和防止皮肤衰老,预防感冒等作用。', 'sections': [{'title': '简介', 'content': '红色食品富含番茄红素、胡萝卜素、铁和部分氨基酸,是优质蛋白质、碳水化合物、膳食纤维、B族维生素和多种无机盐的重要来源,可以弥补粳米、白面中的营养缺失。经常食用红色食品,可以进一步提高对主食中营养的利用率,山植等食品还有治疗癌症的功效。被称为“红色生力军。”营养学家认为,红色蔬果最典型的优势在于它们都是富含天然铁质的食物,例如我们常吃的樱桃、大枣等都是贫血患者的天然良药,也适合女性经期失血后的滋补。所以,红色蔬果,女人尽可放心多吃。红色食品中还含有致病微生物的“杀手”——巨噬细胞,可以有效地抵御感冒病毒等微生物,增强人体抵抗感冒的能力。  红色食品\n在所有的果蔬当中,名声最好的莫过于苹果。西方有“One apple a day,keeps the doctors away.”的说法,因为苹果性情温和,含有各种维生素和微量元素,是所有的水果中最接近完美的一个。\n还有一种说法:红色食品是相对于绿色食品而言的,指对人体有害的食品,如各种有毒有害、腐败变质、添加非食用物质的食品。红色食品危害人体健康乃至生命安全,对人体健康亮起了红灯,应当大力查处。'}, {'title': '作用', 'content': '这些食品中富含β-胡萝卜素和维生素A,对孩子上皮组织和呼吸道粘膜有很强的保护作用,可提高预防感冒的能力。\n假如你生来体质较弱,易受感冒病毒的困扰,或者已经被感冒缠上了,红色食品会助你一臂之力,天生具有促进人体健康卫士之一的巨噬细胞活力的功能,巨噬细胞乃是感冒病毒等致病微生物的“杀手”,其活力增强了,感冒病毒自然难以在人体内立足,更谈不上生长繁殖了。至于颜色较辣椒稍浅一些的胡萝卜,所含的胡萝卜素可在体内转化为维生素A,发挥护卫人体上皮组织如呼吸道黏膜的作用,常食之同样可以增强人体抗御感冒的能力

In [7]:
with open(bd_baike_563w_file, 'r', encoding='utf-8') as f:
    
    while True:
        line = f.readline()
        if not line: break

        item = ujson.loads(line)
        cur_txt = []
       
        cur_txt.append( f"{item['title']}：{item['summary']}")

        for section in item['sections']:
            cur_txt.append( f"{section['title']}：{section['content']}")
        
        # 添加 eos token
        cur_txt.append(eos_token)

        # normalize 处理\u3000 \xa0，全角转半角
        baike_items.append( normalize('NFKC', ''.join(cur_txt)) )

        if len(baike_items) % batch_size == 0:

            chunk_data = split_txt_cropus_to_chunk_data(baike_items)
            tb = pa.Table.from_arrays([chunk_data], names=['text'])

            file_name = f'./data/baike_chunk_320_5.6M_{batch_cnt}.parquet'
            pq.write_table(table=tb, where=file_name, row_group_size=50000, )

            print(f"save to {file_name}")

            batch_cnt += 1
            baike_items = []

    if len(baike_items) > 0:
        chunk_data = split_txt_cropus_to_chunk_data(baike_items)
        tb = pa.Table.from_arrays([chunk_data], names=['text'])

        file_name = f'./data/baike_chunk_320_5.6M_{batch_cnt}.parquet'
        pq.write_table(table=tb, where=file_name, row_group_size=50000, )

        print(f"save to {file_name}")

        batch_cnt += 1
        baike_items = []

save to ./data/baike_chunk_320_5.6M_0.parquet
save to ./data/baike_chunk_320_5.6M_1.parquet
save to ./data/baike_chunk_320_5.6M_2.parquet
save to ./data/baike_chunk_320_5.6M_3.parquet
save to ./data/baike_chunk_320_5.6M_4.parquet
save to ./data/baike_chunk_320_5.6M_5.parquet


In [8]:
file_list = [
    f'./data/baike_chunk_320_5.6M_{batch_cnt}.parquet' for batch_cnt in range(6)
]

line_cnt = 0 
for file in file_list:
    pf = pq.read_table(file)
    line_cnt += pf.num_rows

print(f"bake all lines: {line_cnt}")

bake all lines: 20777087


In [9]:
chunk_data[20: 25]

[')\n室内设计(研)     时尚管理(研)佛罗伦萨珠宝设计学院\nLe Art Orafe Jewellery School珠宝设计\n[EOS]阳光舜城:None阳光舜城简介:舜耕山下8.71平方公里,拥有6000亩树林,2000亩社区绿化,青山绿水,文脉悠久。中城--中央商业区,北城--生态山庄,南城—都市生活区。北依济南千佛山,东邻蚰蜒山风景区,西临金鸡岭游览区,南接外环路,群山环绕。为由济南人民政府委托三联集团开发建设的山东省最大的住宅产业项目,山东省重点工程。阳光舜城锦绣苑:地处阳光舜城南城核心区域,建筑面积18万平方米。锦绣苑南望远山,北依蝎子山,西临龟山,东邻鲁能足球俱乐部训练基地,处于济南千佛山的风景名胜区域,山体、植被',
 '鲁能足球俱乐部训练基地,处于济南千佛山的风景名胜区域,山体、植被、人工绿化情况较好,是济南市适于居住、环境最好的地区之一。项目与舜耕路、旅游路等多条城市主干道相接,交通便捷。 \n锦绣苑周围山势连绵起伏,4000亩原生森林相伴,四季常青。项目临近兴济河,上风上水,规划设计中结合优美的自然山水环境,通过户外空间的组织,打造一种新城市主义下成熟而有活力的邻里关系,营造社区归属感,构筑环境优雅、服务完善的社会单元。项目规划设计全部由板式高层和板式小高层组成,内部景观设计借助周边天然绿色山体,通过居住区内大面积植被与之相呼应,兴济河的贯穿,为居住区水景的营造增添了无限活力。阳光舜城锦绣苑是典型的新兴都市主义城市,是为中产阶层精心构筑的心灵家园',
 '城锦绣苑是典型的新兴都市主义城市,是为中产阶层精心构筑的心灵家园。周边信息:幼儿园:3之3幼儿园、玉函北区幼儿园、舜玉路幼儿园等\n中小学:舜玉小学、舜耕小学、西藏中学、外国语学校三箭分校、舜耕中学等\n大学:山东财政职工大学、济南大学东校区、山东财政学院等\n商场:银座超市、万家乐超市等\n医院:财政学院医院、联合大学医院、小区内设有门诊\n邮局:舜玉邮政分局、玉函北区邮政所等\n银行:中国银行、商业银行、兴业银行、润丰合作社等\n其他:千福居酒店等\n小区内部配套:各小区内均配备小型花园、社区超市等便民设施依山居:“少数人的豪宅”,度身定做的40余套尊贵府邸,小高层,面积从170平米到200平米,户户南向、室室山景的奢华户型,更加适合上流阶层的',
 '米到200平米,户户南

## 1.3 处理bell指令数据
尝试在预训练阶段加入prompt指令数据，就是尝试在预训练解决加加入部分Sft数据

In [21]:
my_data = []
max_len = 384

In [ ]:
with open('./data/bell_open_source/train_3.5M_CN.json', 'r', encoding='utf-8') as f:
    for line in f:
        item = ujson.loads(line)

        if len(item['conversations']) != 2: continue

        conversation = item['conversations']
        txt = ''
        if conversation[0]['from'] =='human':
            txt = f"{conversation[0]['value']}\n{conversation[1]['value']}"
        else:
            txt = f"{conversation[1]['value']}\n{conversation[0]['value']}"

        if len(txt) >= max_len: continue
        my_data.append(txt)

In [12]:

for file in ['./data/bell_open_source/train_2M_CN.json', './data/bell_open_source/Belle_open_source_1M.json']:
    with open(file, 'r', encoding='utf-8') as f:
        for line in f:
            item = ujson.loads(line)

            if item['input'].strip() != '':
                txt = f"{item['instruction']}\n{item['input']}\n{item['output']}"
            else:
                txt = f"{item['instruction']}\n{item['output']}"
            
            if len(txt) == 0 or len(txt) >= max_len: continue
            my_data.append(
                    txt
            )

In [14]:
print(len(my_data), my_data[0:5])


3033566 ['请输入一段描述有关海洋保护的情景对话。\nPerson A: 哇，这个海滩真美啊！\nPerson B: 是啊，可惜现在海洋污染越来越严重了。\nPerson A: 对啊，以前来海边能看到很多海星、海螺等生物，现在好难看到了。\nPerson B: 这些可爱的生物其实都是海洋生态系统的重要组成部分，如果它们消失了，整个生态系统都会受影响。\nPerson A: 我们该怎么做才能保护海洋呢？\nPerson B: 首先，我们要遵守海滩的相关规定，不要随便乱扔垃圾。其次，减少使用一次性塑料制品，比如塑料袋、吸管等，这些东西会进入海洋，危害海洋生物。最后，我们可以积极参与各种海洋保护组织，呼吁更多人加入到保护海洋的行动中来。\nPerson A: 听起来保护海洋是一项重要的任务，我们一定要做好自己的份内工作。\nPerson B: 是的，每个人都能为保护海洋贡献自己的一份力量。', '为我寻找5个值得信任的汽车保险公司\n1. State Farm\n2. Geico\n3. Allstate\n4. Progressive\n5. Farmers Insurance', 'A公司去年亏损了500万美元，今年净利润增长了50%，今年的净利润是多少？\n今年净利润为750万美元', '介绍最适合初学者的编程语言\nPython是最适合初学者学习的编程语言之一。Python具有以下优点：\n1. 简单易学：Python拥有简单易懂的语法和基本代码结构，可以使初学者更快地掌握编程基础。\n2. 应用广泛：Python可以用于很多不同的任务和领域，例如数据科学、人工智能、机器学习、网络编程等。\n3. 强大的社区支持：Python具有庞大的开发者社区，可以获得丰富的资源和支持。\n4. 多资源学习：Python的学习资源丰富，包括图书、视频、课程等。\n总之，对于初学者来说，Python是一门非常好的编程语言，它容易入门，应用广泛，有强大的社区支持和丰富的学习资源。', '以下是一道小学数学题： \n有一条直线，上面有 6 只蚂蚁，其中 4 只向左走，2 只向右走。问：向右走的蚂蚁数量占比是多少？\nA. 25%\nB. 33.3%\nC. 50%\nD. 66.6%\n回答该问题。\n向右走的蚂蚁数量占比为两只蚂蚁中向右走的蚂蚁数量与总蚂蚁数量的比值。\n先

In [18]:
tb = pa.Table.from_arrays([my_data], names=['text'])
# compression='GZIP'
pq.write_table(table=tb, where='./data/bell_pretrain_3M.parquet', row_group_size=20480, data_page_size=20480, )

# 2. 处理sft阶段数据

In [6]:
lines = []
with open('./data/sft_0.8M_CN.json', 'r', encoding='utf-8') as f:
    for line in f:
        item = ujson.loads(line)

        txt = f"{item['instruction']}{item['output']}"
        
        if len(txt) == 0 or len(txt) >= 320: continue
        lines.append(
                item
        )

In [7]:
print(len(lines))

726475


In [8]:
tb = pa.Table.from_pylist(lines)
# compression='GZIP'
pq.write_table(table=tb, where='./data/sft_train_data.parquet', row_group_size=20480, data_page_size=20480, )